In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from collections import Counter

In [2]:
no_text_forecasts = pd.read_pickle("Experiments/unseeded_models/forecasts/STraTS_20-124_4-28-2_forecast.pkl")
notext_forecasts = no_text_forecasts[0]
notext_var_map = list(no_text_forecasts[1].keys())

print("loading STraTS-notext data and forecast")
with open("./data/mimic_notext.pkl", "rb") as pfile:
    raw_data = pickle.load(pfile)

notext_test = pd.read_pickle("./data/mimic_notext_for_eval.pkl")


loading STraTS-notext data and forecast


In [5]:
notext_test

variable,ts_ind,hour,ALP,ALT,AST,Age,Albumin,Albumin 25%,Albumin 5%,Amiodarone,...,Total CO2,Ultrafiltrate,Unknown,Urine,Vancomycin,Vasopressin,WBC,Weight,pH Blood,pH Urine
0,2,0.000000,NaN,NaN,NaN,46.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0.533333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.921578,NaN,NaN
2,2,0.783333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1.033333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.230176,NaN,NaN,NaN
4,2,1.283333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2808194,49403,15.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.093791,NaN,NaN,NaN,NaN,NaN,NaN
2808195,49403,16.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2808196,49403,17.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.310568,NaN,NaN,NaN,NaN,NaN,NaN
2808197,49403,18.850000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#ID und Sepsis Label Paar
full_IDs_labels = raw_data[1][["ts_ind","SUBJECT_ID", "in_hospital_sepsis"]]
full_IDs_labels
pos_IDs = full_IDs_labels["ts_ind"].loc[full_IDs_labels["in_hospital_sepsis"]==1]
pos_IDs

0            0
7            7
22       43456
37          29
60          49
         ...  
49521    41647
49558    41676
49591    41703
49595    41707
49610    41716
Name: ts_ind, Length: 5252, dtype: int64

In [18]:
labels = notext_test.copy()
labels
labels["sepsis_label"] = np.where(labels["ts_ind"].isin(pos_IDs), 1,0)


Counter({0: 2119366, 1: 688833})

In [19]:
labels["sepsis_label"]

0          0
1          0
2          0
3          0
4          0
          ..
2808194    0
2808195    0
2808196    0
2808197    0
2808198    0
Name: sepsis_label, Length: 2808199, dtype: int32

In [7]:
#Test IDs
notext_forecast_test_IDs_labels = notext_forecasts[["ts_ind","SUBJECT_ID", "sepsis_label"]]
notext_forecast_test_IDs_labels

,ts_ind,SUBJECT_ID,sepsis_label
0,2,271,0
1,3,272,0
2,8,281,0
3,11,284,0
4,20,294,0
...,...,...,...
9652,49393,56179,0
9653,49394,47998,0
9654,49397,103,0
9655,49400,83967,0


In [ ]:
notext_mimic = raw_data[0]
notext_mimic = notext_mimic.loc[~notext_mimic["ts_ind"].isin(notext_forecast_test_IDs_labels["ts_ind"])]
notext = notext_mimic.pivot_table(index=['ts_ind', "hour"], columns='variable', values='value')
notext = notext.reset_index()
notext.to_pickle("mimic_notext_for_class_eval.pkl")

MemoryError: Unable to allocate 452. MiB for an array with shape (59292333,) and data type int64

In [8]:
#Rest IDs
#full_mimic = raw_data[0]
test = notext_test.loc[notext_test["ts_ind"].isin(notext_forecast_test_IDs_labels["ts_ind"])]
non_test = notext[~notext["ts_ind"].isin(notext_forecast_test_IDs_labels["ts_ind"])]




NameError: name 'notext' is not defined

In [7]:
non_test

variable,ts_ind,hour,ALP,ALT,AST,Age,Albumin,Albumin 25%,Albumin 5%,Amiodarone,...,Total CO2,Ultrafiltrate,Unknown,Urine,Vancomycin,Vasopressin,WBC,Weight,pH Blood,pH Urine


In [9]:
len(np.unique(notext["ts_ind"]))

9657

In [14]:
from sklearn.model_selection import GroupShuffleSplit
from tslearn.svm import TimeSeriesSVC
gss = GroupShuffleSplit(n_splits=1, train_size=.7, random_state=1)



##for testing before committing to actually generate the long data..
non_test = test



#Classifier für Full Train/Test data Performance-> Full Observed
# X rows * 133 features full hours
X = non_test[notext_var_map]
# Y 
Y = pd.DataFrame(np.where(non_test["ts_ind"].isin(pos_IDs), 1,0))
groups = non_test["ts_ind"]
for i, (train_index, test_index) in enumerate(gss.split(X, Y, groups)):
    X_train = X.iloc[train_index]
    Y_train = Y.iloc[train_index]
    X_test = X.iloc[test_index]
    Y_test = Y.iloc[test_index]


c:\Users\pablo\miniconda3\envs\microsoft_sepsis\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [16]:
clf = TimeSeriesSVC(C=1.0, kernel="gak", n_jobs=-1)
clf.fit(X_train, Y_train)
clf.score(X_test, Y_test)

#dann nur noch evaluaten auf 4h+ forecasts??





#Classifier 4h Train 4h Test data Performance -> 4h Observed
#non_test_4h = non_test.loc[non_test["hour"]<=4]
#X_4h = non_test_4h[notext_var_map]
#Y_4h = np.where(X_4h["ts_ind"].isin(pos_IDs), 1,0)
#Classifier 28h Train, 4h+24Foreacst Performance -> Forecast

c:\Users\pablo\miniconda3\envs\microsoft_sepsis\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MemoryError: Unable to allocate 1.97 GiB for an array with shape (1990081, 133, 1) and data type float64